In [ ]:
import os, sys, datetime, json, numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import tikzplotlib as tkz

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print(f"Current path: {os.getcwd()}")
sys.path.append('./')

from supervisedlearning.approximation import ApproximatedHeteroskedasticGaussianProcessRegressorRBF

from utils import make_json_serializable

In [ ]:
n_samples_plt = 1000
x_plt = np.linspace(0, 1, n_samples_plt)

def exp_reward_values(xvect):
    return np.cos(xvect * 6.28 - 1)

dateformat = "(%Y-%b-%d %Ih%Mm%p)"
save_path = "results/supervised/approximation/"


# TESTING THE FITTING QUALITY FOR DIFFERENT KERNEL LENGTHS
N_test = 100
reg = 0.01
x_vect = np.linspace(0, 1, N_test)
y_vect = exp_reward_values(x_vect)
x_vect = x_vect.reshape(N_test, 1)

plt.figure()
plt.plot(x_vect.ravel(), y_vect, "--", linewidth=4, label="RealFun")

for kernel_L in [10]:

    regressor = ApproximatedHeteroskedasticGaussianProcessRegressorRBF(kernel_L, reg)
    regressor.load_data(x_vect, y_vect)
    y_hat, y_sigma = regressor.compute(x_plt.reshape(-1, 1))
    plt.plot(x_plt.ravel(), y_hat, label=f"L={kernel_L}")
    plt.fill_between(x_plt.ravel(), y_hat - y_sigma, y_hat + y_sigma, alpha=0.3)
                
plt.legend()

In [ ]:
kernel_L_lst = [10]
epsilon_lst = [None, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
regularization_lst = [1, 5, 10, 50, 100, 500, 1000] # for the first two kinds of tests
# regularization_lst = [2, 4, 6, 8, 10, 20, 40, 60, 80, 100] # for the last kind of tests
N_lst = [200, 400, 600, 800, 1000]

assert epsilon_lst[0] is None, "The first element of epsilon_lst should be None"
results = {}

for N in N_lst:
    print(f"Processing N = {N}...")
    results[N] = {}
    np.random.seed(0)
    x_vect = np.linspace(0, 1, N)
    y_vect = exp_reward_values(x_vect)
    x_vect = x_vect.reshape(N, 1)
        
    for kernel_L in kernel_L_lst:
        results[N][kernel_L] = {}

        for regularization in regularization_lst:
            print(f"    Processing regularization = {regularization}...")
            results[N][kernel_L][regularization] = {}
            y_hat, y_sigma = {}, {}

            for epsilon in epsilon_lst:
                results[N][kernel_L][regularization][epsilon] = {}
                regressor = ApproximatedHeteroskedasticGaussianProcessRegressorRBF(kernel_L, regularization, epsilon=epsilon)
                regressor.load_data(x_vect, y_vect)
                y_hat[epsilon], y_sigma[epsilon] = regressor.compute(x_plt.reshape(-1, 1))
                results[N][kernel_L][regularization][epsilon]["errorinfnorm"] = np.round(max(np.abs(y_hat[epsilon] - y_hat[None])), 5)
                results[N][kernel_L][regularization][epsilon]["errorinfnormvar"] = np.round(max(np.abs(y_sigma[epsilon] - y_sigma[None])), 5)
                results[N][kernel_L][regularization][epsilon]["nsample"] = regressor.cover_x.shape[0] if epsilon is not None else None

for N in N_lst:
    for kernel_L in kernel_L_lst:
        for regularization in regularization_lst:
            for epsilon in epsilon_lst:
                print(f"N = {N} - kernel_L = {kernel_L} - $\lambda$ = {regularization} - $\epsilon$ = {epsilon} \t Error = {results[N][kernel_L][regularization][epsilon]['errorinfnorm']}    \t Cover Size = {results[N][kernel_L][regularization][epsilon]['nsample']}")

serializable_dict = make_json_serializable(results)
complete_path_json = f"{save_path}N{N_lst}_kernelL{kernel_L_lst}_reg{regularization_lst}_eps{epsilon_lst}_{datetime.datetime.now().strftime(dateformat)}.json"
with open(complete_path_json, "w", encoding="utf-8") as filejson:
    json.dump(serializable_dict, filejson, ensure_ascii=False, indent=4)

In [ ]:
save = True

N_vect = np.array(N_lst)
epsilon_vect = np.array(epsilon_lst[1:])

# reg fixed
# forall N forall kernel_L
# for every lambda plot infnorm-error varying epsilon

norm = plt.Normalize(-1, np.log(max(regularization_lst)))
cmap = cm.get_cmap("viridis_r")

for N in N_lst:
    for kernel_L in kernel_L_lst:
        plt.figure()
        plt.title(f"N = {N} - kernel_L = {kernel_L}")
        for regularization in regularization_lst:
            err_vect = -1 * np.ones((len(epsilon_lst)-1))
            for eps_i, epsilon in enumerate(epsilon_lst[1:]): 
                err_vect[eps_i] = results[N][kernel_L][regularization][epsilon]['errorinfnorm']
                color = cmap(norm(np.log(regularization)))
            plt.plot(epsilon_vect, err_vect, "o-", label=f"$\lambda$={regularization}", color=color)
        plt.xlim([0, 1])
        plt.xlabel("$\epsilon$")
        plt.ylabel("Error w.r.t $\widehat{\mu}$") 
        plt.legend(loc="best")
        if save:
            complete_path = f"{save_path}mean_N{N}_L{kernel_L}_reglst{regularization_lst}_epslst{epsilon_lst}_{datetime.datetime.now().strftime(dateformat)}"
            plt.savefig(f"{complete_path}.jpg")
            tkz.save(f"{complete_path}.tex")


for N in N_lst:
    for kernel_L in kernel_L_lst:
        plt.figure()
        plt.title(f"N = {N} - kernel_L = {kernel_L}")
        for regularization in regularization_lst:
            err_vect = -1 * np.ones((len(epsilon_lst)-1))
            for eps_i, epsilon in enumerate(epsilon_lst[1:]): 
                err_vect[eps_i] = results[N][kernel_L][regularization][epsilon]['errorinfnormvar']
                color = cmap(norm(np.log(regularization)))
            plt.plot(epsilon_vect, err_vect, "o-", label=f"$\lambda$={regularization}", color=color)
        plt.xlim([0, 1])
        plt.xlabel("$\epsilon$")
        plt.ylabel("Error w.r.t $\widehat{\sigma}$") 
        plt.legend(loc="best")
        if save:
            complete_path = f"{save_path}var_N{N}_L{kernel_L}_reglst{regularization_lst}_epslst{epsilon_lst}_{datetime.datetime.now().strftime(dateformat)}"
            plt.savefig(f"{complete_path}.jpg")
            tkz.save(f"{complete_path}.tex")


# reg fixed
# forall eps forall kernel_L
# for every lambda plot infnorm-error varying N

norm = plt.Normalize(-1, np.log(max(regularization_lst)))
cmap = cm.get_cmap("viridis_r")

for kernel_L in kernel_L_lst:
    for epsilon in epsilon_lst[1:]:
        plt.figure()
        plt.title(f"$\epsilon$ = {epsilon} - kernel_L = {kernel_L}")
        for regularization in regularization_lst:
            err_vect = -1 * np.ones((len(N_lst)))
            err_vect_postvariance = -1 * np.ones((len(N_lst)))
            for N_i, N in enumerate(N_lst): 
                err_vect[N_i] = results[N][kernel_L][regularization][epsilon]['errorinfnorm']
                err_vect_postvariance[N_i] = results[N][kernel_L][regularization][epsilon]['errorinfnormvar']
            plt.plot(N_vect, err_vect, "o-", label=f"$\lambda$={regularization} - mean", color = cmap(norm(np.log(regularization))))
            plt.plot(N_vect, err_vect_postvariance, "^--", label=f"$\lambda$={regularization} - var", color = cmap(norm(np.log(regularization))))
        # plt.xlim([0, 1])
        plt.xlabel("$N$")
        plt.ylabel("Error w.r.t $\widehat{\mu}$") 
        plt.legend(loc="best")
        if save:
            complete_path = f"{save_path}eps{epsilon}_L{kernel_L}_reglst{regularization_lst}_Nlst{N_lst}_{datetime.datetime.now().strftime(dateformat)}"
            plt.savefig(f"{complete_path}.jpg")
            tkz.save(f"{complete_path}.tex")
            

# N/reg fixed
# forall ratios forall kernel_L
# for every lambda plot infnorm-error varying N

symbols = ['o-', '^-']
norm = plt.Normalize(0, max(N_lst))
cmap = cm.get_cmap("viridis_r")

for kernel_L in kernel_L_lst:
    plt.figure()
    for ratio_i, ratio in enumerate([1/10, 1/100]):
        # plt.title(f"ratio N/reg = {ratio} - kernel_L = {kernel_L}")
        for N in N_lst:
            err_vect = -1 * np.ones((len(epsilon_lst)-1))
            for eps_i, epsilon in enumerate(epsilon_lst[1:]): 
                err_vect[eps_i] = results[N][kernel_L][int(N*ratio)][epsilon]['errorinfnorm']
            color = cmap(norm(N))
            plt.plot(epsilon_vect, err_vect, symbols[ratio_i], label=f"$ N / \\lambda $={ratio} --- $ N $={N}", color=color)
        plt.xlabel("$\\epsilon$")
        plt.ylabel("Error w.r.t $\\widehat{\\mu}$") 
        plt.legend(loc="best")
    if save:
        complete_path = f"{save_path}ratiomeans_L{kernel_L}_Nlst{N_lst}_reglst{regularization_lst}_epslst{epsilon_lst}_{datetime.datetime.now().strftime(dateformat)}"
        plt.savefig(f"{complete_path}.jpg")
        tkz.save(f"{complete_path}.tex")


for kernel_L in kernel_L_lst:
    plt.figure()
    for ratio_i, ratio in enumerate([1/10, 1/100]):
        # plt.title(f"ratio N/reg = {ratio} - kernel_L = {kernel_L}")
        for N in N_lst:
            err_vect = -1 * np.ones((len(epsilon_lst)-1))
            for eps_i, epsilon in enumerate(epsilon_lst[1:]): 
                err_vect[eps_i] = results[N][kernel_L][int(N*ratio)][epsilon]['errorinfnormvar']
            color = cmap(norm(N))
            plt.plot(epsilon_vect, err_vect, symbols[ratio_i], label=f"$ N / \\lambda \\!=\\!{ratio}$ -- $ N \\!=\\!{N}$", color=color)
        plt.xlabel("$\\epsilon$")
        plt.ylabel("Error w.r.t $\\widehat{\\sigma}$") 
        plt.legend(loc="best")
    if save:
        complete_path = f"{save_path}ratiovar_L{kernel_L}_Nlst{N_lst}_reglst{regularization_lst}_epslst{epsilon_lst}_{datetime.datetime.now().strftime(dateformat)}"
        plt.savefig(f"{complete_path}.jpg")
        tkz.save(f"{complete_path}.tex")